# EDA и продуктовый анализ — BMW Global Sales (2010-2024)

Этот ноутбук на русском языке содержит полный EDA-пайплайн, визуализации и продуктовые гипотезы для датасета `BMW Global Sales` (2010–2024). Цель — получить инсайты для продуктовой стратегии и подготовить материалы для презентации.

Файлы данных: `/mnt/data/BMW sales data (2010-2024) (1).csv`

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

%matplotlib inline
plt.rcParams['figure.figsize'] = (10,5)
plt.rcParams['grid.linestyle'] = '--'

DATA_PATH = 'BMW sales data (2010-2024) .csv'
OUT_DIR = Path('/mnt/data/bmw_project_outputs_noteboo')
OUT_DIR.mkdir(parents=True, exist_ok=True)

print('Готово: импорты и директории')

Готово: импорты и директории


In [4]:
# Загрузка данных

df = pd.read_csv(DATA_PATH)
print('Rows, cols:', df.shape)
df.head(7)

Rows, cols: (50000, 11)


,Model,Year,Region,Color,Fuel_Type,Transmission,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Sales_Classification
0,5 Series,2016,Asia,Red,Petrol,Manual,3.5,151748,98740,8300,High
1,i8,2013,North America,Red,Hybrid,Automatic,1.6,121671,79219,3428,Low
2,5 Series,2022,North America,Blue,Petrol,Automatic,4.5,10991,113265,6994,Low
3,X3,2024,Middle East,Blue,Petrol,Automatic,1.7,27255,60971,4047,Low
4,7 Series,2020,South America,Black,Diesel,Manual,2.1,122131,49898,3080,Low
5,5 Series,2017,Middle East,Silver,Diesel,Manual,1.9,171362,42926,1232,Low
6,i8,2022,Europe,White,Diesel,Manual,1.8,196741,55064,7949,High


In [5]:
# Базовая информация и предобработка

df.info()

# Приведём типы, уберём очевидные NA
cols_to_num = ['Year','Price_USD','Sales_Volume']
for c in cols_to_num:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Уберём строки без Year/Price/Sales
df = df.dropna(subset=['Year','Price_USD','Sales_Volume'])
df['Year'] = df['Year'].astype(int)

print('После очистки:', df.shape)

# Простые проверки
print('\nУникальные регионы:', df['Region'].nunique())
print('Уникальных моделей:', df['Model'].nunique())

# Сохранение очищенной копии
df.to_csv(OUT_DIR / 'cleaned_bmw_sales.csv', index=False)
print('Сохранён cleaned_bmw_sales.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Model                 50000 non-null  object 
 1   Year                  50000 non-null  int64  
 2   Region                50000 non-null  object 
 3   Color                 50000 non-null  object 
 4   Fuel_Type             50000 non-null  object 
 5   Transmission          50000 non-null  object 
 6   Engine_Size_L         50000 non-null  float64
 7   Mileage_KM            50000 non-null  int64  
 8   Price_USD             50000 non-null  int64  
 9   Sales_Volume          50000 non-null  int64  
 10  Sales_Classification  50000 non-null  object 
dtypes: float64(1), int64(4), object(6)
memory usage: 4.2+ MB
После очистки: (50000, 11)

Уникальные регионы: 6
Уникальных моделей: 11
Сохранён cleaned_bmw_sales.csv


In [6]:
# Динамика глобальных продаж по годам

sales_per_year = df.groupby('Year', as_index=False)['Sales_Volume'].sum().sort_values('Year')
ax = sales_per_year.plot('Year','Sales_Volume', marker='o', title='Total BMW Sales Volume by Year (2010-2024)')
ax.set_ylabel('Sales Volume')
plt.grid(True)
plt.tight_layout()
plt.savefig(OUT_DIR / 'plot_sales_per_year.png')
sales_per_year.head()

,Year,Sales_Volume
0,2010,16933445
1,2011,16758941
2,2012,16751895
3,2013,16866733
4,2014,16958960


In [7]:
# Продажи по регионам — топ-6

sales_by_region = df.groupby(['Year','Region'], as_index=False)['Sales_Volume'].sum()
# Топ-6 регионов по сумме
top_regions = sales_by_region.groupby('Region')['Sales_Volume'].sum().nlargest(6).index.tolist()
sales_by_region_top = sales_by_region[sales_by_region['Region'].isin(top_regions)]

fig, ax = plt.subplots(figsize=(11,5))
for region in top_regions:
    ser = sales_by_region_top[sales_by_region_top['Region']==region].sort_values('Year')
    ax.plot(ser['Year'], ser['Sales_Volume'], marker='o', label=region)
ax.set_title('Sales by Year — Top 6 Regions')
ax.set_ylabel('Sales Volume')
ax.legend()
ax.grid(True)
plt.tight_layout()
plt.savefig(OUT_DIR / 'plot_sales_by_region_top6.png')

sales_by_region_top.head()

,Year,Region,Sales_Volume
0,2010,Africa,2855044
1,2010,Asia,2907671
2,2010,Europe,2775123
3,2010,Middle East,2623155
4,2010,North America,2876099


In [8]:
# Топ-модели по суммарным продажам

sales_by_model = df.groupby('Model', as_index=False)['Sales_Volume'].sum().sort_values('Sales_Volume', ascending=False)
TOP_N = 10
plt.figure(figsize=(8,5))
plt.barh(sales_by_model.head(TOP_N).sort_values('Sales_Volume', ascending=True)['Model'],
         sales_by_model.head(TOP_N).sort_values('Sales_Volume', ascending=True)['Sales_Volume'])
plt.title(f'Top {TOP_N} BMW Models by Total Sales (2010-2024)')
plt.xlabel('Total Sales Volume')
plt.tight_layout()
plt.savefig(OUT_DIR / 'plot_top10_models.png')

sales_by_model.head(15)

,Model,Sales_Volume
2,7 Series,23786466
10,i8,23423891
5,X1,23406060
0,3 Series,23281303
9,i3,23133849
1,5 Series,23097519
4,M5,22779688
6,X3,22745529
7,X5,22709749
8,X6,22661986


In [9]:
# Динамика по типам топлива

fuel_by_year = df.groupby(['Year','Fuel_Type'], as_index=False)['Sales_Volume'].sum()
fuel_pivot = fuel_by_year.pivot(index='Year', columns='Fuel_Type', values='Sales_Volume').fillna(0)

plt.figure(figsize=(10,5))
plt.stackplot(fuel_pivot.index, [fuel_pivot[col] for col in fuel_pivot.columns], labels=fuel_pivot.columns)
plt.title('Fuel Type Sales Volume by Year (stacked)')
plt.xlabel('Year')
plt.ylabel('Sales Volume')
plt.legend(loc='upper left')
plt.tight_layout()
plt.savefig(OUT_DIR / 'plot_fuel_share_by_year.png')

fuel_pivot.tail()

Fuel_Type,Diesel,Electric,Hybrid,Petrol
Year,,,,
2020,4152177,3962571,4032718,4163377
2021,4137203,4239630,4331469,4176364
2022,4474126,4383912,4687463,4375445
2023,3842804,4305554,4013825,4106471
2024,4356475,4290700,4647195,4233484


In [10]:
# Распределение цен и Price vs Sales

plt.figure(figsize=(8,4))
plt.hist(df['Price_USD'], bins=40)
plt.title('Price Distribution (USD)')
plt.xlabel('Price_USD')
plt.ylabel('Count')
plt.tight_layout()
plt.savefig(OUT_DIR / 'hist_price.png')

# агрегация по model-year для scatter
price_sales = df.groupby(['Model','Year'], as_index=False).agg({'Price_USD':'mean','Sales_Volume':'sum'})
plt.figure(figsize=(7,5))
plt.scatter(price_sales['Price_USD'], price_sales['Sales_Volume'], alpha=0.6)
plt.title('Price vs Sales Volume (model-year aggregated)')
plt.xlabel('Average Price (USD)')
plt.ylabel('Sales Volume')
plt.grid(True)
plt.tight_layout()
plt.savefig(OUT_DIR / 'plot_price_vs_sales.png')

price_sales.sample(8)

,Model,Year,Price_USD,Sales_Volume
97,X3,2017,74728.089965,1461422
99,X3,2019,74817.667763,1482528
150,i8,2010,75148.132911,1626565
52,M3,2017,71921.503425,1474399
44,7 Series,2024,75333.679878,1686209
113,X5,2018,77729.878689,1497265
125,X6,2015,72683.262411,1405983
78,X1,2013,73559.691617,1730904


In [11]:
# BCG-like матрица по моделям (рост vs market share)

model_year_sales = df.groupby(['Model','Year'], as_index=False)['Sales_Volume'].sum()
pivot = model_year_sales.pivot(index='Model', columns='Year', values='Sales_Volume').fillna(0)

years = sorted(df['Year'].unique())
start_year = years[0]; end_year = years[-1]; n = end_year - start_year

def compute_cagr_proxy(row):
    start = row.get(start_year, 0)
    end = row.get(end_year, 0)
    if start>0 and end>0:
        return (end/start)**(1/n)-1
    vals = row.values
    slope = np.polyfit(range(len(vals)), vals, 1)[0]
    return slope/(np.mean(vals)+1e-9)

bcg_df = pivot.apply(compute_cagr_proxy, axis=1).to_frame('CAGR').join(pivot.sum(axis=1).rename('Total_Sales'))
bcg_df['Market_Share'] = bcg_df['Total_Sales'] / bcg_df['Total_Sales'].sum()
bcg_df = bcg_df.reset_index().sort_values('Total_Sales', ascending=False)

plt.figure(figsize=(8,6))
plt.scatter(bcg_df['CAGR'], bcg_df['Market_Share'], s=(bcg_df['Total_Sales'] / bcg_df['Total_Sales'].max())*500 + 20, alpha=0.7)
plt.axvline(bcg_df['CAGR'].median(), linestyle='--')
plt.axhline(bcg_df['Market_Share'].median(), linestyle='--')
plt.title('BCG-like matrix: Model Growth (CAGR proxy) vs Market Share')
plt.xlabel('Growth (CAGR proxy)')
plt.ylabel('Market Share')
for i,row in bcg_df.head(15).iterrows():
    plt.annotate(row['Model'], (row['CAGR'], row['Market_Share']), fontsize=8, xytext=(3,3), textcoords='offset points')
plt.tight_layout()
plt.savefig(OUT_DIR / 'plot_bcg_matrix.png')

bcg_df.head(12)

,Model,CAGR,Total_Sales,Market_Share
2,7 Series,0.013996,23786466,0.093878
10,i8,-0.001684,23423891,0.092447
5,X1,0.000643,23406060,0.092377
0,3 Series,-0.006144,23281303,0.091885
9,i3,0.000089,23133849,0.091303
1,5 Series,0.009411,23097519,0.091159
4,M5,0.001684,22779688,0.089905
6,X3,0.003525,22745529,0.089770
7,X5,-0.002750,22709749,0.089629
8,X6,0.016974,22661986,0.089440


## Ключевые числовые выводы (сводка)

- Рассчитать CAGR глобальных продаж, по типам топлива и по регионам (2010→2024).
- Выделить топ-модели и их динамику.

---

## Продуктовые гипотезы (включены в ноутбук)

1. Фокус на Middle East и Europe даст лучший прирост.
2. Инвестиции в Hybrid дают более быстрый ROI, чем полный переход на EV прямо сейчас.
3. Пересмотреть портфель моделей: развивать X-серии и 7 Series, уменьшить фокус на моделях с отрицательным ростом (например, 3 Series в наших данных).

In [12]:
# Расчёт некоторых числовых показателей для вставки в отчёт

# Total CAGR
sales_per_year = df.groupby('Year', as_index=False)['Sales_Volume'].sum().sort_values('Year')
start_val = int(sales_per_year.iloc[0]['Sales_Volume'])
end_val = int(sales_per_year.iloc[-1]['Sales_Volume'])
start_year = int(sales_per_year.iloc[0]['Year'])
end_year = int(sales_per_year.iloc[-1]['Year'])
n = end_year - start_year

total_cagr = (end_val/start_val)**(1/n)-1

# fuel cagr
fuel_by_year = df.groupby(['Year','Fuel_Type'], as_index=False)['Sales_Volume'].sum()
fuel_pivot = fuel_by_year.pivot(index='Year', columns='Fuel_Type', values='Sales_Volume').fillna(0)
fuel_cagr = {}
for col in fuel_pivot.columns:
    s = fuel_pivot[col].iloc[0]
    e = fuel_pivot[col].iloc[-1]
    fuel_cagr[col] = {'start':int(s),'end':int(e),'cagr': (e/s)**(1/n)-1 if s>0 else None}

# region cagr for top6
sales_by_region = df.groupby(['Year','Region'], as_index=False)['Sales_Volume'].sum()
top_regions = sales_by_region.groupby('Region')['Sales_Volume'].sum().nlargest(6).index.tolist()
region_stats = {}
for region in top_regions:
    ser = sales_by_region[sales_by_region['Region']==region].sort_values('Year')
    s = int(ser.iloc[0]['Sales_Volume'])
    e = int(ser.iloc[-1]['Sales_Volume'])
    region_stats[region] = {'start':s,'end':e,'cagr': (e/s)**(1/n)-1 if s>0 else None}

summary = {'total_cagr': total_cagr, 'start_year':start_year, 'end_year':end_year, 'start_total':start_val, 'end_total':end_val}

summary, fuel_cagr, region_stats

({'total_cagr': 0.0024673684089155934,
  'start_year': 2010,
  'end_year': 2024,
  'start_total': 16933445,
  'end_total': 17527854},
 {'Diesel': {'start': 4086808,
   'end': 4356475,
   'cagr': np.float64(0.004574647530575016)},
  'Electric': {'start': 4205554,
   'end': 4290700,
   'cagr': np.float64(0.0014327294580194216)},
  'Hybrid': {'start': 4415611,
   'end': 4647195,
   'cagr': np.float64(0.0036579307765030045)},
  'Petrol': {'start': 4225472,
   'end': 4233484,
   'cagr': np.float64(0.00013531802384258995)}},
 {'Asia': {'start': 2907671, 'end': 3080909, 'cagr': 0.00414228896170421},
  'Europe': {'start': 2775123, 'end': 3033044, 'cagr': 0.0063681671838748954},
  'North America': {'start': 2876099,
   'end': 2862275,
   'cagr': -0.00034409070898600014},
  'Middle East': {'start': 2623155,
   'end': 2943091,
   'cagr': 0.008254064049526777},
  'Africa': {'start': 2855044, 'end': 2805506, 'cagr': -0.0012494573263245323},
  'South America': {'start': 2896353,
   'end': 2803029,
 

In [13]:
# Сохранение агрегатов и артефактов

sales_per_year.to_csv(OUT_DIR / 'sales_per_year.csv', index=False)
sales_by_region_top.to_csv(OUT_DIR / 'sales_by_region_year_top6.csv', index=False)
sales_by_model.head(50).to_csv(OUT_DIR / 'sales_by_model_top50.csv', index=False)
bcg_df.to_csv(OUT_DIR / 'bcg_by_model.csv', index=False)
price_sales.to_csv(OUT_DIR / 'price_sales_model_year.csv', index=False)

print('Агрегаты сохранены в', OUT_DIR)


Агрегаты сохранены в \mnt\data\bmw_project_outputs_notebook


## Выводы и дальнейшие шаги

- В ноутбуке есть все необходимые визуализации и агрегаты для создания презентации.
- Дальше можно: 1) добавить прогноз (Prophet), 2) сделать интерактивный Streamlit-дэшборд, 3) подготовить PPT с выводами.

---

**Файлы, которые будут сгенерированы и сохранены:** в папке `/mnt/data/bmw_project_outputs_notebook` — графики и CSV с агрегатами.
